In [1]:
##. Enable CPU fallback when needed
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

# Practice using PyTorch to build a penguin classifier

## Setup

Loading in the packages needed, and the data

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
##. Importing the Palmer Penguins Dataset is made easy thanks to
##. https://github.com/mcnakhaee/palmerpenguins
from palmerpenguins import load_penguins

In [3]:
##. Make use of a GPU or MPS (Apple) if one is available.
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: mps


In [4]:
##. Loading the penguin data
df = load_penguins()
df = df.dropna().copy()
df.head(10)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,male,2007
6,Adelie,Torgersen,38.9,17.8,181.0,3625.0,female,2007
7,Adelie,Torgersen,39.2,19.6,195.0,4675.0,male,2007
12,Adelie,Torgersen,41.1,17.6,182.0,3200.0,female,2007
13,Adelie,Torgersen,38.6,21.2,191.0,3800.0,male,2007
14,Adelie,Torgersen,34.6,21.1,198.0,4400.0,male,2007


## Preparing the data for training the model

We want our data to be loaded into PyTorch tensors so we can build and train a pytorch model

In [5]:
##. Prepare to pre-process the target labels
target_label_encoder = preprocessing.LabelEncoder()

##. Get the variable data into numpy format
x_vars = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
x = df[x_vars].values

##. Encode the target variables as integers
y = target_label_encoder.fit_transform(df['species'])

##. Keep track of the actual labels for later
species_labels = target_label_encoder.classes_

##. Make the variables and targets into PyTorch tensors
x = torch.tensor(x, device=device, dtype=torch.float32)
y = torch.tensor(y, device=device, dtype=torch.long)

## Building a model
For the sake of practice, this is a simple sequential neural network with two hidden layers.

I'm using `CrossEntropyLoss` as my loss criterion because this is a multi-classification task.

In [6]:
## Build a sequental model using PyTorch
model = nn.Sequential(
    ##. Input layer
    ##.     Inputs: Match the shape of variables
    ##.     Outputs: 200
    ##.     Activation: ReLU
    nn.Linear(x.shape[1], 200),
    nn.ReLU(),
    ##. Hidden layer 1: 
    ##.     Inputs: 200 (matches output of previous layer)
    ##.     Outputs: 50
    ##.     Activation: ReLU
    nn.Linear(200,50),
    nn.ReLU(),
    ##. Hidden layer 2: 
    ##.     Inputs: 50 (matches output of previous layer)
    ##.     Outputs: 25
    ##.     Activation: ReLU
    nn.Linear(50,25),
    nn.ReLU(),
    ##. Output Layer
    ##.    Inputs: 12
    ##.    Outputs: Match the number of output classes
    ##.    Activation: LogSoftmax (determined by the choice of criterion, see below)
    nn.Linear(25,len(species_labels))
)

##. Send the model to the device
##. Info: https://stackoverflow.com/questions/59560043/what-is-the-difference-between-model-todevice-and-model-model-todevice
model = model.to(device)

##. Deine the loss criterion
##. Note: CrossEntropyLoss combines nn.LogSoftmax() and nn.NLLLoss() so don't use Softmax in the model
criterion = nn.CrossEntropyLoss()

##. Define and Optimizer and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

## Training the model
Here we train the model, and improve it over a number of epochs using gradient descent.

In [7]:
##. Switch model into training mode
model.train()

for epoch in range(5000):
    ## Zero out gradient between epochs
    optimizer.zero_grad()
    ## Compute the output of this epoch of the model on x
    out = model(x)
    ## Compute the loss at this epoch
    loss = criterion(out, y)
    ## Backpropogate the gradient of the loss function at x
    loss.backward()
    ## Step the model parameters in the direction of the gradient
    optimizer.step()

    ## Print out progress every 250 epochs
    if epoch % 250 == 0:
        print(f"Epoch {epoch}, loss: {loss.item()}")

Epoch 0, loss: 60.247196197509766
Epoch 250, loss: 0.8229854702949524
Epoch 500, loss: 0.7576754689216614
Epoch 750, loss: 0.70575350522995
Epoch 1000, loss: 0.6589992642402649
Epoch 1250, loss: 0.6120021939277649
Epoch 1500, loss: 0.5639272928237915
Epoch 1750, loss: 0.51280677318573
Epoch 2000, loss: 0.4596543312072754
Epoch 2250, loss: 0.40567904710769653
Epoch 2500, loss: 0.35448983311653137
Epoch 2750, loss: 0.3093382716178894
Epoch 3000, loss: 0.26653164625167847
Epoch 3250, loss: 0.23120832443237305
Epoch 3500, loss: 0.197788268327713
Epoch 3750, loss: 0.1699349582195282
Epoch 4000, loss: 0.14575821161270142
Epoch 4250, loss: 0.12545262277126312
Epoch 4500, loss: 0.10858079046010971
Epoch 4750, loss: 0.10102535039186478


## Evaluating the model

In [8]:
##. Swtich model into evaluation mode
model.eval()

##. Use the model to predict the species on our data
pred = model(x)

##. Grab the predicted classes for scoring purposes
_, predict_classes = torch.max(pred, 1)

##. Calculate the accuracy score for our predictions
correct = accuracy_score(y.cpu().detach(), predict_classes.cpu().detach())
print(f"Accuracy: {correct}")

Accuracy: 0.975975975975976


Not too shabby!